# AutoGen入門

このコードサンプルでは、[AutoGen](https://aka.ms/ai-agents-beginners/autogen) マルチエージェント会話フレームワークを使用して基本的なエージェントを作成します。

## 📚 このノートブックで学習できること

### 🎯 主要な学習目標
- **AutoGen**フレームワークの基本概念
- **マルチエージェント会話**システムの構築
- **Azure AI Chat Completion Client**との連携
- **GitHub Models**を使用したLLMアクセス

### 🛠️ 実装する技術スタック
- **AutoGen**: Microsoft製マルチエージェント会話フレームワーク
- **Azure AI Chat Completion Client**: 統一エンドポイントアクセス
- **GitHub Models**: gpt-4o-miniモデルへのアクセス
- **非同期処理**: ストリーミング応答の実装

### 💡 実用的なスキル
- エージェント間の会話設計
- Azure AI Services統合
- マルチエージェントワークフローの実装

## 必要なパッケージのインストール

以下のコードセルは、このサンプルに必要なパッケージをインストールします。パッケージが既にインストールされている場合は、このセルをスキップできます。

In [1]:
%pip install autogen-agentchat autogen-core autogen-ext[azure] azure-ai-inference python-dotenv --quiet --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 必要なPythonパッケージのインポート

In [3]:
import asyncio
import os
from typing import Any, Dict

from azure.core.credentials import AzureKeyCredential
from autogen_core.models import UserMessage
from autogen_core import CancellationToken

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient

from dotenv import load_dotenv

## クライアントの作成

このサンプルでは、LLMへのアクセスに[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

GitHub ModelsのベースURLに使用される`Azure AI Chat Completion Client`を使用するために、AutoGen内の`AzureAIInferenceChatCompletionClient`コネクタを使用します。他のモデルプロバイダーでAutoGenを使用するための[他の利用可能なコネクタ](https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/tutorial/models.html)もあります。

In [4]:
# 環境変数を読み込み
load_dotenv()

# AutoGenモデルクライアントを作成（GitHub Models使用）
model_client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # GitHub設定で個人アクセストークン（PAT）を生成する必要があります
    # PATトークンの作成手順: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

# 簡単なテスト実行
result = await model_client.create([UserMessage(content="フランスの首都は何ですか？", source="user")])
print(result)

c:\VSCode - Work\ai-agents-for-beginners\ai-agents-for-beginners\myenv\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


finish_reason='stop' content='フランスの首都はパリです。' usage=RequestUsage(prompt_tokens=18, completion_tokens=12) cached=False logprobs=None thought=None


## エージェントの定義

`client`を設定し、動作することを確認したので、`AssistantAgent`を作成しましょう。各エージェントには以下を割り当てることができます：

**name** - マルチエージェントフローで参照する際に便利な短縮名
**model_client** - 前のステップで作成したクライアント
**tools** - エージェントがタスクを完了するために使用できる利用可能なツール
**system_message** - LLMのタスク、動作、トーンを定義するメタプロンプト

system_messageを変更して、LLMがどのように応答するかを確認できます。`tools`については第4課で説明します。

In [5]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[],
    system_message="あなたは素晴らしいバケーションを計画する旅行代理店です",
)

## エージェントの実行

以下の関数はエージェントを実行します。`on_messages`メソッドを使用して、新しいメッセージでエージェントの状態を更新します。

この場合、ユーザーからの新しいメッセージ`"素晴らしい晴れたバケーションを計画してください"`で状態を更新します。

LLMがどのように異なって応答するかを確認するために、メッセージの内容を変更できます。

In [6]:
from IPython.display import display, HTML

async def assistant_run():
    # クエリを定義
    user_query = "素晴らしい晴れたバケーションを計画してください"

    # HTML出力の構築開始
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>ユーザー:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # エージェントの応答を実行
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # エージェントの応答をHTMLに追加
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>アシスタント:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # フォーマットされたHTMLを表示
    display(HTML(html_output))

# 関数を実行
await assistant_run()

## 🎓 学習のまとめ

### ✅ このサンプルで習得したスキル
- **AutoGen**の基本的な使用方法
- **AssistantAgent**の作成と設定
- **GitHub Models**との連携
- **基本的なエージェント実行**パターン

### 🔄 AutoGenの主要概念
1. **AssistantAgent**: 特定のタスクを実行するエージェント
2. **model_client**: LLMサービスへの接続
3. **system_message**: エージェントの動作を定義するプロンプト
4. **on_messages**: エージェントとの対話メソッド

### 🚀 次のステップ
このサンプルの目標は、異なるAgenticパターンを実装する際に後の追加コードサンプルで使用する手順を示すことです。

後続のレッスンでは以下を学習します：
- より複雑なエージェントワークフロー
- マルチエージェントシステム
- ツールとプラグインの統合
- 高度な会話パターン

### 🔧 トラブルシューティング
- **GITHUB_TOKEN**: `.env`ファイルで正しく設定されているか確認
- **パッケージインストール**: 最新のAutoGenパッケージが必要
- **モデルアクセス**: GitHub Modelsへの適切なアクセス権限の確認